In [7]:
from __future__ import annotations

```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\providecommand{\tightlist}{%
  \setlength{\itemsep}{0pt}\setlength{\parskip}{0pt}}

```

```{=latex}
\title{Iterate, Iterate, Iterate}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}
```

```{=latex}
\frame{\titlepage}
```

## Iteration protocol

### Iterable vs. Iterator

In [13]:
from typing import TypeVar, Protocol, Any, Generic
from dataclasses import dataclass

T = TypeVar("T")


```{=latex}
\begin{frame}[fragile]
\frametitle{Iterable or Iterator}
```

In [8]:
class Iterable(Protocol[T]):
    def __iter__(self) -> Iterator[T]:
        ...

```{=latex}
\pause
```

In [10]:
class Iterator(Iterable[T]):
    def __next__(self) -> T:
        """Raises StopIterationError"""

```{=latex}
\pause

The \verb|__iter__| method returns \verb|self|
```

```{=latex}
\end{frame}
```

### `__iter__`

```{=latex}
\begin{frame}[fragile]
\frametitle{Making it iterable}
```

In [15]:
@dataclass(frozen=True)
class TwoInts:
    thing_1: int
    thing_2: int
    
    def __iter__(self) -> Iterator[T]:
        return iter([self.thing_1, self.thing_2])

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Making it iterable}
```

In [18]:
dec = TwoInts(1, 10)
iter_dec = iter(dec)
print("thing_1", iter_dec.__next__())
print("thing_2", iter_dec.__next__())

thing_1 1
thing_2 10


```{=latex}
\end{frame}
```

### `__next__`

```{=latex}
\begin{frame}[fragile]
\frametitle{Moving Forward}
```

In [24]:
class TwoIntIter:
    def __init__(self) -> None:
        self.things = [1, 10]
    
    def __next__(self) -> int:
        try:
            return self.things.pop(0)
        except IndexError:
            raise StopIteration() from None

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Moving Forward}
```

In [26]:
iter_dec = TwoIntIter()
print(iter_dec.__next__())
print(iter_dec.__next__())

1
10


```{=latex}
\end{frame}
```

### `next`

```{=latex}
\begin{frame}[fragile]
\frametitle{Moving Forward: Usage}
```

In [28]:
iter_dec = iter([1, 10])
print(next(iter_dec))
print(next(iter_dec))
try:
    next(iter_dec)
except Exception as exc:
    print("Stopped", repr(exc))

1
10
Stopped StopIteration()


```{=latex}
\end{frame}
```

### `for`

```{=latex}
\begin{frame}[fragile]
\frametitle{for: Using iterators}
```

In [35]:
class TwoIntIter:
    def __init__(self) -> None:
        self.things = [1, 10]
    
    def __iter__(self) -> Iterator[int]:
        return self

    def __next__(self) -> int:
        try:
            ret_value = self.things.pop(0)
        except IndexError:
            print("__next__: End of iteration")
            raise StopIteration() from None
        print("__next__: Returning", ret_value)
        return ret_value

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{for: Using iterators}
```

In [42]:
iter_dec = TwoIntIter()
print("for: Before")
for val in iter_dec:
    print("for: Got", val)
print("for: After")

for: Before
__next__: Returning 1
for: Got 1
__next__: Returning 10
for: Got 10
__next__: End of iteration
for: After


```{=latex}
\end{frame}
```

### Sequence constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Sequence constructors: Using iterators}
```

In [38]:
res = list(TwoIntIter())
print("Result is", res)

__next__: Returning 1
__next__: Returning 10
__next__: End of iteration
Result is [1, 10]


```{=latex}
\end{frame}
```

## Generators

### Generator vs. Iterator

```{=latex}
\begin{frame}[fragile]
\frametitle{Generator vs. Iterator}

Iterator: Protocol \pause

Annoying to implement (state) \pause

Generator: Mechanism to build iterators

\end{frame}
```

### `yield`

```{=latex}
\begin{frame}[fragile]
\frametitle{Generators and yield}
```

In [39]:
def two_int_iter():
    print("Begin")
    yield 1
    print("Middle")
    yield 10
    print("End")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Generators and yield}
```

In [43]:
iter_dec = two_int_iter()
print("for: Before")
for val in iter_dec:
    print("for: Got", val)
print("for: After")

for: Before
Begin
for: Got 1
Middle
for: Got 10
End
for: After


```{=latex}
\end{frame}
```

### `yield from`

```{=latex}
\begin{frame}[fragile]
\frametitle{yield from}
```

In [44]:
def three_int_iter():
    yield from two_int_iter()
    yield 100

list(three_int_iter())

Begin
Middle
End


[1, 10, 100]

```{=latex}
\end{frame}
```

### `deque(maxlen=0)`

```{=latex}
\begin{frame}[fragile]
\frametitle{Mindless consumption}
```

In [45]:
from collections import deque

deque(two_int_iter(), maxlen=0)

Begin
Middle
End


deque([], maxlen=0)

```{=latex}
\end{frame}
```

## Combining iterators

### `enumerate`

```{=latex}
\begin{frame}[fragile]
\frametitle{enumerate}
```

In [49]:
list(enumerate(iter([1, 10, 100])))

[(0, 1), (1, 10), (2, 100)]

```{=latex}
\end{frame}
```

### `map`

```{=latex}
\begin{frame}[fragile]
\frametitle{map}
```

In [50]:
from operator import add
from functools import partial

list(map(
    partial(add, 1),
    iter([10, 20]),
))

[11, 21]

```{=latex}
\end{frame}
```

### `filter`

```{=latex}
\begin{frame}[fragile]
\frametitle{filter}
```

In [53]:
from operator import ge
from functools import partial

list(filter(
    partial(ge, 5),
    iter([1, 10, 3]),
))

[1, 3]

```{=latex}
\end{frame}
```

### `zip`

```{=latex}
\begin{frame}[fragile]
\frametitle{zip}
```

In [54]:
list(zip(
    iter([10, 100]),
    iter([2, 4]),
))

[(10, 2), (100, 4)]

```{=latex}
\end{frame}
```

### Comprehensions

```{=latex}
\begin{frame}[Comprehensions]
\frametitle{zip}
```

In [56]:
double_plus_one = (
    2 * a + 1
    for a in
    iter([1, 10])
)

list(double_plus_one)

[3, 21]

```{=latex}
\end{frame}
```

## `itertools`

In [57]:
import itertools

### `chain`

```{=latex}
\begin{frame}[fragile]
\frametitle{chain}
```

In [58]:
list(
    itertools.chain(
        iter([1, 2, 3]),
        iter([4, 5]),
    )
)

[1, 2, 3, 4, 5]

```{=latex}
\end{frame}
```

### `islice`

```{=latex}
\begin{frame}[fragile]
\frametitle{islice}
```

In [60]:
list(itertools.islice(two_int_iter(), 0, 1))

Begin


[1]

```{=latex}
\end{frame}
```

### `count`

```{=latex}
\begin{frame}[fragile]
\frametitle{count}
```

In [62]:
list(itertools.islice(itertools.count(), 0, 5))

[0, 1, 2, 3, 4]

```{=latex}
\end{frame}
```

## `more_itertools`

In [2]:
import more_itertools

### `chunked`

```{=latex}
\begin{frame}[fragile]
\frametitle{chunked}
```

In [5]:
list(more_itertools.chunked(iter([0, 1, 2, 3]), 2))

[[0, 1], [2, 3]]

```{=latex}
\end{frame}
```

### `distribute`

```{=latex}
\begin{frame}[fragile]
\frametitle{distribute}
```

In [7]:
evens, odds = more_itertools.distribute(2, iter([0, 1, 2, 3]))
print(list(evens))
print(list(odds))

[0, 2]
[1, 3]


```{=latex}
\end{frame}
```

### `peekable`

```{=latex}
\begin{frame}[fragile]
\frametitle{peekable}
```

In [11]:
some_nums = more_itertools.peekable(iter([0, 1, 2, 3, 4]))

In [12]:
for x in some_nums:
    if some_nums.peek() == 3:
        break
    print(x)

0
1


```{=latex}
\end{frame}
```

### `windowed`

```{=latex}
\begin{frame}[fragile]
\frametitle{windowed}
```

In [19]:
samples = more_itertools.windowed(
    iter([0, 3, 7, 10, 12]),
    2
)

In [20]:
for start, end in samples:
    print(end - start)

3
4
3
2


```{=latex}
\end{frame}
```

## Summary

### Iterators and Generators

```{=latex}
\begin{frame}[fragile]
\frametitle{Iterators and Generators}

\pause

Iterators: Useful interface

\pause

Generators: Useful way to implement interface

\end{frame}
```

### Algebra for Just-in-time

```{=latex}
\begin{frame}[fragile]
\frametitle{Algebra of Just-in-time}

\pause

Functions that accept and return iterators

\pause

Composable just-in-time tools

\end{frame}
```

### Ecosystem

```{=latex}
\begin{frame}[fragile]
\frametitle{Ecosystem}

\pause

Research

\pause

Consume

\pause

Contribute

\end{frame}
```

```{=latex}
\end{document}
```